In [1]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=8a9f2a41b96c9513510e2c94e2d7afe4997c54817c6be471959096c5368af4ea
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [46]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML
import os
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import os
import matplotlib.pyplot as plt
from ta.trend import SMAIndicator, EMAIndicator
from ta.momentum import RSIIndicator
from ta.momentum import StochasticOscillator
from ta.trend import MACD
from ta.volatility import BollingerBands
import glob
from joblib import Parallel, delayed

In [2]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 31.4 MB/s eta 0:00:00


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.action_chains import ActionChains
from datetime import date

In [15]:
requests.packages.urllib3.disable_warnings()
import warnings
warnings.filterwarnings('ignore')

In [31]:
url="https://www.mse.mk/mk/issuers/free-market"
base_url="https://www.mse.mk"
answer = requests.get(url)

In [32]:
from selenium.webdriver.support.ui import Select

In [40]:
def Filter2(url):
  request = requests.get(url)
  text=request.text
  soup=BeautifulSoup(text,'html.parser')
  body = soup.select('.col-md-12 a')
  links = [a['href'] for a in body if a.has_attr('href')]
  return links[0]

In [41]:
def get_last_date_csv(csv_file):
    filename = f'{csv_file}.csv'
    if os.path.exists(filename):
        df = pd.read_csv('/content/'+filename)
        if not df.empty:
            df['Датум'] = pd.to_datetime(df['Датум'], errors='coerce')
            last_date = df['Датум'].max()
            if pd.notna(last_date):
                return last_date.strftime("%d.%m.%Y")
        else:
            return None
    else:
        columns = ['Датум', 'Цена на последна трансакција', 'Мак.', 'Мин.', 'Просечна цена', '%пром.', 'Количина', 'Промет во БЕСТ во денари', 'Вкупен промет во денари']
        df = pd.DataFrame(columns=columns)
        df.to_csv(filename, index=False)

In [42]:
def get_data(csv_file, url, start_date, end_date):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    data = []
    start_date_picker=driver.find_element(By.ID,'FromDate')
    start_date_picker.clear()
    start_date_picker.send_keys(start_date)
    end_date_picker=driver.find_element(By.ID,'ToDate')
    end_date_picker.clear()
    end_date_picker.send_keys(end_date)
    button=driver.find_element(By.CLASS_NAME,'btn-primary-sm')
    driver.execute_script('arguments[0].scrollIntoView();',button)
    button.click()
    time.sleep(1)
    request = driver.page_source
    soup = BeautifulSoup(request, 'html.parser')
    body = soup.select('tbody tr')
    for b in body:
      cols = b.find_all('td')
      if len(cols) >= 9:
          data.append({
                    "Датум": cols[0].text.strip(),
                    "Цена на последна трансакција": cols[1].text.strip(),
                    "Мак.": cols[2].text.strip(),
                    "Мин.": cols[3].text.strip(),
                    "Просечна цена": cols[4].text.strip(),
                    "%пром.": cols[5].text.strip(),
                    "Количина": cols[6].text.strip(),
                    "Промет во БЕСТ во денари": cols[7].text.strip()+',00',
                    "Вкупен промет во денари": cols[8].text.strip()+',00'
                })
    driver.quit()

    return data

In [43]:
def process_firm(link):
    name = Filter2(base_url + link)
    if '#' in name:
        return
    parts = name.split('/')
    name = parts[4]
    data = []

    if get_last_date_csv(name) is None:
        start_date = datetime(2014, 1, 1).strftime('%d.%m.%Y')
        end_date = datetime(2014, 12, 31).strftime('%d.%m.%Y')
        data += get_data(name, base_url + Filter2(base_url + link), start_date, end_date)
    else:
        start_date = get_last_date_csv(name)
        year_start = start_date.split('.')
        today = str(date.today())
        year_end = today.split('-')
        end_year = int(year_end[0])
        start_year = int(year_start[2])

        for year in range(start_year, end_year + 1):
            if year == end_year:
                end_month = int(year_end[1])
                end_day = int(year_end[2])
                end_date = datetime(year, end_month, end_day).strftime('%d.%m.%Y')
            else:
                end_date = datetime(year, 12, 31).strftime('%d.%m.%Y')
            start_date = datetime(year, 1, 1).strftime('%d.%m.%Y')
            data += get_data(name, base_url + Filter2(base_url + link), start_date, end_date)

    df = pd.DataFrame(data)
    df.to_csv(f'/content/{name}.csv')

In [47]:
def main():
    text = answer.text
    soup = BeautifulSoup(text, 'html.parser')
    body = soup.select("tbody tr a")
    links = [a['href'] for a in body if a.has_attr('href')]
    Parallel(n_jobs=8)(delayed(process_firm)(link) for link in links)

In [ ]:
if __name__ == "__main__":
    main()

In [24]:
from pickle import NONE
def load_data(file_path):
    df = pd.read_csv(file_path)
    df['Датум'] = pd.to_datetime(df['Датум'], format='%d.%m.%Y')
    df.sort_values('Датум', inplace=True)
    if 'Цена на последна трансакција' not in df.columns or df['Цена на последна трансакција'].isna().all():
        return None
    df['Цена'] = df['Цена на последна трансакција'].str.replace(',', '').astype(float)
    return df

In [25]:
def resample_data(df, interval):
    df = df.set_index('Датум').resample(interval).last()
    df['Цена'] = df['Цена'].interpolate()
    return df.reset_index()

In [26]:
def calculate_oscillators(df):
    oscillators = []

    # Calculate RSI
    rsi = RSIIndicator(df['Цена'], window=14).rsi()
    rsi_decision = np.where(rsi < 30, 'Buy', np.where(rsi > 70, 'Sell', 'Hold'))
    oscillators.append(("RSI", rsi, rsi_decision))

    # Calculate Stochastic Oscillator
    stoch = StochasticOscillator(df['Цена'], df['Цена'], df['Цена'], window=14).stoch()
    stoch_decision = np.where(stoch < 20, 'Buy', np.where(stoch > 80, 'Sell', 'Hold'))
    oscillators.append(("Stochastic", stoch, stoch_decision))

    # Calculate MACD
    macd = MACD(df['Цена']).macd()
    macd_decision = np.where(macd > 0, 'Buy', 'Sell')
    oscillators.append(("MACD", macd, macd_decision))

    # Calculate Bollinger Bands High
    bollinger_high = BollingerBands(df['Цена']).bollinger_hband()
    bollinger_high_decision = np.where((bollinger_high.notna()) & (df['Цена'] > bollinger_high), 'Sell', 'Hold')
    oscillators.append(("Bollinger High", bollinger_high, bollinger_high_decision))

    # Calculate Bollinger Bands Low
    bollinger_low = BollingerBands(df['Цена']).bollinger_lband()
    bollinger_low_decision = np.where((bollinger_low.notna()) & (df['Цена'] < bollinger_low), 'Buy', 'Hold')
    oscillators.append(("Bollinger Low", bollinger_low, bollinger_low_decision))

    return oscillators

In [28]:
def calculate_moving_averages(df):
    mas = []

    # Calculate SMA 20
    sma_20 = SMAIndicator(df['Цена'], window=20).sma_indicator()
    sma_20_decision = np.where(sma_20.notna() & (df['Цена'] > sma_20), 'Buy', 'Sell')
    mas.append(("SMA_20", sma_20, sma_20_decision))

    # Calculate SMA 50
    sma_50 = SMAIndicator(df['Цена'], window=50).sma_indicator()
    sma_50_decision = np.where(sma_50.notna() & (df['Цена'] > sma_50), 'Buy', 'Sell')
    mas.append(("SMA_50", sma_50, sma_50_decision))

    # Calculate EMA 20
    ema_20 = EMAIndicator(df['Цена'], window=20).ema_indicator()
    ema_20_decision = np.where(ema_20.notna() & (df['Цена'] > ema_20), 'Buy', 'Sell')
    mas.append(("EMA_20", ema_20, ema_20_decision))

    # Calculate EMA 50
    ema_50 = EMAIndicator(df['Цена'], window=50).ema_indicator()
    ema_50_decision = np.where(ema_50.notna() & (df['Цена'] > ema_50), 'Buy', 'Sell')
    mas.append(("EMA_50", ema_50, ema_50_decision))

    # Calculate EMA 200
    ema_200 = EMAIndicator(df['Цена'], window=200).ema_indicator()
    ema_200_decision = np.where(ema_200.notna() & (df['Цена'] > ema_200), 'Buy', 'Sell')
    mas.append(("EMA_200", ema_200, ema_200_decision))

    return mas

In [27]:
def generate_dataframe(data_list):
    names, values, decisions = [], [], []
    for name, value_series, decision_series in data_list:
        if len(value_series.dropna()) == 0 or len(decision_series) == 0:
            continue
        names.append(name)
        values.append(value_series.dropna().iloc[-1])  # Safely get the last value
        decisions.append(decision_series[-1])  # Safely get the last decision

    result = pd.DataFrame({
        'Name': names,
        'Value': values,
        'Decision': decisions
    })
    return result

In [29]:
def process_firm_data(file_path, interval):
    df = load_data(file_path)
    if df is None:
        return None, None
    # Resample data for the interval
    df = resample_data(df, interval)

    # Calculate oscillators and MAs
    oscillators = calculate_oscillators(df)
    mas = calculate_moving_averages(df)

    # Generate final dataframes
    osc_df = generate_dataframe(oscillators)
    ma_df = generate_dataframe(mas)

    return osc_df, ma_df

In [30]:
folder_path = '/content'
file_paths = os.listdir(folder_path)
intervals = {'1D': 'daily', '1W': 'weekly', '1M': 'monthly'}

for file_name in file_paths:
    if not file_name.endswith('.csv'):
        continue
    full_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(full_path)
    if df.empty:
        continue
    for interval, interval_name in intervals.items():
        osc_df, ma_df = process_firm_data(full_path, interval)
        if osc_df is None or ma_df is None:
            continue
        osc_df.to_csv(f"/content/oscillators_{interval_name}_{file_name}", index=False)
        ma_df.to_csv(f"/content/moving_averages_{interval_name}_{file_name}", index=False)